## Trying out different libraries and algorithms

Let's first import Python/Sage bindings to compute embeddings using different libraries and algorithms.

In [2]:
from ffisom import *

Lets define an extension finite field with different underlying libraries:
* Sage uses PARI/GP by default;
* we added a wrapper for FLINT implementation.

In [16]:
p = 7
n = 237
q = p**n
k = GF(q, name='z')
k_flint = GF_flint(p, k.modulus(), name='z')

Here comes data of importance to the algorithms:
* o is the degree of the auxiliary extension needed for cyclotomic Rains
* c is the degree of the cyclotomic extension needed for Kummer type algorithms

In [18]:
o, _ = find_root_order(p, [n, n], n, verbose=False)
c = Mod(p,n).multiplicative_order()
print o, c

1 78


#### Algebraic groups algorithms

We cannot test Magma implementation of cyclotomic Rains here, but we can test ours.

* with the default PARI/GP implementation for finite field arithmetic:

In [11]:
%time a, b = find_gens_cyclorains(k, k)
print a.minpoly() == b.minpoly()

CPU times: user 43 ms, sys: 2 ms, total: 45 ms
Wall time: 43.3 ms
True


* or FLINT implementation:

In [10]:
%time a, b = find_gens_cyclorains(k_flint, k_flint)
print a.minpoly() == b.minpoly()

CPU times: user 34 ms, sys: 4 ms, total: 38 ms
Wall time: 35.9 ms
True


As o == 2, the conic Rains algorithm does not apply here, but we can try the elliptic variation.

* if PARI/GP is used for finite field arithmetic, then Sage is used for elliptic curve arithmetic:

In [21]:
%time a, b = find_gens_ellrains(k, k)
print a.minpoly() == b.minpoly()

CPU times: user 1.44 s, sys: 1 ms, total: 1.44 s
Wall time: 1.44 s
True


* if FLINT is, then the C code from ellmul is used:

In [22]:
%time a, b = find_gens_ellrains(k_flint, k_flint)
print a.minpoly() == b.minpoly()

CPU times: user 391 ms, sys: 2 ms, total: 393 ms
Wall time: 388 ms
True


#### Kummer type algorithms

Let's try PARI/GP implementation of Lenstra/Allombert:

In [12]:
%time a, b = find_gens_pari(k, k)
print a.minpoly() == b.minpoly()

CPU times: user 2.95 s, sys: 0 ns, total: 2.95 s
Wall time: 2.95 s
True


And finally our C++ implementations of variations on Lenstra/Allombert:

In [15]:
for i, algo in enumerate(kummer_algolist[2*(c==1):-2*(c==1)-1]):
    print kummer_namelist[2*(c==1)+i]
    %time a, b = find_gens_kummer(k_flint, k_flint, n, algo)
    print a.minpoly() == b.minpoly()

LINALG_CYCLO
CPU times: user 584 ms, sys: 0 ns, total: 584 ms
Wall time: 583 ms
True
LINALG_ONLY
CPU times: user 211 ms, sys: 0 ns, total: 211 ms
Wall time: 211 ms
True
LINALG
CPU times: user 217 ms, sys: 0 ns, total: 217 ms
Wall time: 217 ms
True
MODCOMP
CPU times: user 217 ms, sys: 0 ns, total: 217 ms
Wall time: 217 ms
True
COFACTOR
CPU times: user 190 ms, sys: 0 ns, total: 190 ms
Wall time: 189 ms
True
ITERFROB
CPU times: user 193 ms, sys: 0 ns, total: 193 ms
Wall time: 193 ms
True
MPE
CPU times: user 233 ms, sys: 0 ns, total: 233 ms
Wall time: 233 ms
True


### Benchmarking

All of the above can be automated to get performance data using functions from the benchmark module.
The p, n, o, c stuff is as above and are repeated on the timing line where it is followed by timings for :
* Magma version of cyclotomic Rains
* our Sage/C version of cyclotomic Rains
* our Sage/C version of conic Rains
* our Sage/C version of elliptic Rains
* PARI/GP version of Lenstra/Allombert
* our C++ version of Lenstra/Allombert using linear algebra over the cyclotomic extension
* our C++ version of Lenstra/Allombert using linear algebra even for Frobenius computation à la PARI/GP
* our C++ version of Lenstra/Allombert using linear algebra for kernel elements but modular composition for Frobenius computation
* our C++ version of Lenstra/Allombert using modular composition and a trace like formula for kernel elements
* our C++ version of Lenstra/Allombert using modular composition and the cofactor trick for kernel elements
* our C++ version of Lenstra/Allombert using naive iterated Frobenius with modular exponentiation
* our C++ version of Lenstra/Allombert using iterated Frobenius with multipoint evaluation

In [23]:
from benchmark import *
benchmark(pbound=[3, 10], nbound=[3,10], prime=True, skip_magma=True)

p = 3, n = 5, (o = 1, c = 4)                                                   
3 5 (1, 4) 0 0.00104029178619 0 0 0.000214171409607 0.000252461433411 0.000206804275513 0.000239109992981 0.00023558139801 0.000199675559998 0.000184798240662 0.000219798088074
p = 3, n = 7, (o = 4, c = 6)                                                   
3 7 (4, 6) 0 0.0182385921478 0 0 0.000329279899597 0.000423884391785 0.000290179252625 0.00036346912384 0.000365328788757 0.000275325775146 0.000277233123779 0.000323891639709
p = 5, n = 3, (o = 2, c = 2)                                                   
5 3 (2, 2) 0 0.00926465988159 0.00410797595978 0.00565741062164 0.000153589248657 8.82625579834e-05 7.37428665161e-05 8.65697860718e-05 8.50915908813e-05 7.74383544922e-05 6.81638717651e-05 7.14063644409e-05
p = 5, n = 7, (o = 2, c = 6)                                                   
5 7 (2, 6) 0 0.0130973339081 0.00559575557709 0.00507645606995 0.00050151348114 0.000452399253845 0.000354504585266 0.0